In [1]:
from datasets import Dataset
import pandas as pd
import instructor
from openai import OpenAI

client = instructor.from_openai(OpenAI())

dataset_path = "nl-logql-dataset-classified-metric-with-variables"
dataset = Dataset.load_from_disk(dataset_path)
df = dataset.to_pandas()

In [85]:
pd.set_option("display.max_colwidth", 1000)

In [3]:
# show all rows where variables is None
df["variables"].isna()

0     False
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Name: variables, Length: 100, dtype: bool

In [4]:
df.columns

Index(['application', 'id', 'question', 'logql_query', 'query_explanation',
       'query_result', 'category', 'log_category_result', 'line_filter',
       'label_filter', 'metric_category_result', 'metric_category',
       'variables'],
      dtype='object')

In [6]:
print(df.sample(n=1).to_json())

{"application":{"12":"openstack"},"id":{"12":14},"question":{"12":"What is the latest usage status of image 0673dd71-34c5-4fbb-86c4-40623fbe45b4?"},"logql_query":{"12":"(last_over_time({application=\"openstack\", log_file_type=\"nova-compute\"}\n|= \"0673dd71-34c5-4fbb-86c4-40623fbe45b4\"\n|= \"in use:\"\n| regexp `image (?P<image_id>[^ ]+) at \\((?P<image_path>[^)]+)\\): in use: on this node (?P<local_use>\\d+) local, (?P<other_nodes>\\d+) on other nodes sharing this instance storage`\n| line_format \"{{.image_id}} has {{.local_use}}\"\n| unwrap local_use\n[40d]))"},"query_explanation":{"12":"1\n{application=\"openstack\", log_file_type=\"nova-compute\"}\nFetch all log lines matching label filters.\n2\n<expr> |= `0673dd71-34c5-4fbb-86c4-40623fbe45b4`\nReturn log lines that contain string 0673dd71-34c5-4fbb-86c4-40623fbe45b4.\n\n3\n<expr> |= `in use:`\nReturn log lines that contain string in use:.\n\n4\n<expr> | regexp `image (?P<image_id>[^ ]+) at \\((?P<image_path>[^)]+)\\): in use: 

In [4]:
df

,application,id,question,logql_query,query_explanation,query_result,category,log_category_result,line_filter,label_filter,metric_category_result,metric_category,variables
0,openstack,2,How long did it take to spawn instance 3edec1e...,"{application=""openstack"", log_file_type=""nova-...",bla,3edec1e4-9678-4a3a-a21b-a145a4ee5e61 took 20.58,,"{'chain_of_thought': 'Analyzing the query, it ...",multiple line filters,multiple log stream selectors,"{'categories': None, 'chain_of_thought': 'The ...",None,"[instance_id, time_in_sec]"
1,openstack,3,What was the total time taken to build instanc...,"{application=""openstack"", log_file_type=""nova-...","1. {application=""openstack"", log_file_type=""no...",21.38,,{'chain_of_thought': 'The user query submits u...,multiple line filters,multiple log stream selectors,"{'categories': None, 'chain_of_thought': 'This...",None,"[instance_id, time_in_sec]"
2,openstack,4,What was the total time taken to build instanc...,"{application=""openstack"", log_file_type=""nova-...","1\n{application=""openstack"", log_file_type=""no...",vcpu: 0.00 VCPU used out of 16 VCPU\ndisk: 0.0...,,{'chain_of_thought': 'For the log query in que...,multiple line filters,multiple log stream selectors,"{'categories': None, 'chain_of_thought': 'The ...",None,"[instance_id, resource]"
3,openstack,5,What is the vCPU usage for compute node cp-1.s...,max by (node) (\n max_over_time(\n {applic...,"1\n{application=""openstack"", log_file_type=""no...",<graph>\ngraph with plot of used_vcpus across ...,,{'chain_of_thought': 'The query contains three...,multiple line filters,multiple log stream selectors,"{'categories': ['unwrapped_range_aggregation',...","[unwrapped_range_aggregation, built_in_range_a...","[compute_node, time_in_days]"
4,openstack,6,What is the RAM usage for compute node cp-1.sl...,max by (node) (\n max_over_time(\n {applic...,"1\n{application=""openstack"", log_file_type=""no...",<graph>\\ngraph with plot of used_vcpus across...,,{'chain_of_thought': 'This query has three lab...,multiple line filters,multiple log stream selectors,"{'categories': ['built_in_range_aggregation', ...","[built_in_range_aggregation, unwrapped_range_a...","[compute_node, time_in_days]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,openstack,97,What is the total size of all active base files?,sum by (component) (\n count_over_time({appli...,"1\n{application=""openstack"", component=""nova.v...",12.0k\n<graph>,Image and File Management,{'chain_of_thought': 'The log query contains t...,single line filter,multiple log stream selectors,"{'categories': ['built_in_range_aggregation', ...","[built_in_range_aggregation, log_range_aggrega...",[time_in_hours]
96,openstack,98,What is the average response time for GET requ...,"avg(\n avg_over_time(\n {application=""open...","\n1. `{application=""openstack"", log_file_type=...",0.264\n<graph>,API Performance and Requests,{'chain_of_thought': 'The query specifies two ...,multiple line filters,multiple log stream selectors,"{'categories': ['built_in_range_aggregation', ...","[built_in_range_aggregation, unwrapped_range_a...","[http_method, url_endpoint, time_in_hour]"
97,openstack,99,How many POST requests to /v2/{tenant_id}/os-s...,"sum(count_over_time({application=""openstack""}\...","1. `{application=""openstack"", log_file_type=""n...",0,API Performance and Requests,{'chain_of_thought': 'The query provided uses ...,multiple line filters,single log stream selector,"{'categories': ['log_range_aggregation', 'buil...","[log_range_aggregation, built_in_range_aggrega...","[http_method, url_endpoint, status_code, time_..."
98,openstack,100,What is the 95th percentile response time for ...,"quantile_over_time(0.95,\n {application=""open...","1. `{application=""openstack"", log_file_type=""n...",0.23,API Performance and Requests,"{'chain_of_thought': 'In this query, there are...",multiple line filters,multiple log stream selectors,{'categories': ['unwrapped_range_aggregation']...,[unwrapped_

## Cleaning

- Clean the variables column
- Add `row_variables` column for variables pertaining to a row
- Add `application_variables` column for each application

- For each application, we need to have the log templates stores
- For each application we need to have the NL string of the application vars args

In [22]:
from datasets import Dataset
import pandas as pd
import instructor
from openai import OpenAI

from dotenv import load_dotenv

load_dotenv(".env")

client = instructor.from_openai(OpenAI())

dataset_path = "nl-logql-dataset-classified-metric-with-variables"
dataset = Dataset.load_from_disk(dataset_path)
df = dataset.to_pandas()

### Read OpenStack Templates

In [23]:
openstack_templates_df = pd.read_csv(
    "../logs/OpenStack/OpenStack_full.log_templates.csv"
)
openstack_full_log_df = pd.read_csv(
    "../logs/OpenStack/OpenStack_full.log_structured.csv"
)
openstack_templates_df = openstack_templates_df.sort_values(
    by="Occurrences", ascending=False
)

In [24]:
openstack_templates = []

for index, row in openstack_templates_df.iterrows():
    event_id = row["EventId"]
    event_template = row["EventTemplate"]
    content = openstack_full_log_df[openstack_full_log_df["EventId"] == event_id].iloc[
        0
    ]["Content"]
    openstack_templates.append(
        {"EventId": event_id, "EventTemplate": event_template, "Content": content}
    )

openstack_df = pd.DataFrame(openstack_templates)

In [25]:
openstack_application_var = ["application"]

### Read OpenSSH Templates

In [26]:
openssh_templates_df = pd.read_csv("../logs/OpenSSH/OpenSSH_full.log_templates.csv")
openssh_full_log_df = pd.read_csv("../logs/OpenSSH/OpenSSH_full.log_structured.csv")
openssh_templates_df = openssh_templates_df.sort_values(
    by="Occurrences", ascending=False
)

In [27]:
openssh_templates = []

for index, row in openssh_templates_df.iterrows():
    event_id = row["EventId"]
    event_template = row["EventTemplate"]
    content = openssh_full_log_df[openssh_full_log_df["EventId"] == event_id].iloc[0][
        "Content"
    ]
    openssh_templates.append(
        {"EventId": event_id, "EventTemplate": event_template, "Content": content}
    )

openssh_df = pd.DataFrame(openssh_templates)

In [28]:
openssh_application_var = ["application", "hostname"]

### Read HDFS Templates

In [7]:
hdfs_templates_df = pd.read_csv("../logs/HDFS/HDFS_full.log_templates.csv")
hdfs_full_log_df = pd.read_csv("../logs/HDFS/HDFS_full.log_structured.csv")
hdfs_templates_df = hdfs_templates_df.sort_values(by="Occurrences", ascending=False)

In [8]:
hdfs_templates = []

for index, row in hdfs_templates_df.iterrows():
    event_id = row["EventId"]
    event_template = row["EventTemplate"]
    content = hdfs_full_log_df[hdfs_full_log_df["EventId"] == event_id].iloc[0][
        "Content"
    ]
    hdfs_templates.append(
        {"EventId": event_id, "EventTemplate": event_template, "Content": content}
    )
hdfs_df = pd.DataFrame(hdfs_templates)

In [29]:
hdfs_application_var = ["application"]

*Common Dict*

In [30]:
from typing import Dict, Tuple, List
from pandas import DataFrame

application_dict: Dict[str, Tuple[DataFrame, List]] = {
    "openstack": (openstack_df, openstack_application_var),
    "hdfs": (hdfs_df, hdfs_application_var),
    "openssh": (openssh_df, openssh_application_var),
}

### Pre-process/investigate

In [94]:
import tiktoken


def count_tokens(text: str, encoding_name: str = "cl100k_base"):
    encoding = tiktoken.encoding_for_model("gpt-4o")
    # encoding = tiktoken.get_encoding(encoding_name=encoding_name)
    return len(encoding.encode(text))


def process_token_count(row):
    subset_dict = {
        "id": row.get("id", None),
        "application": row.get("application", None),
        "logql_query": row.get("logql_query", None),
        "variables": row.get("variables", None),
        "query_explanation": row.get("query_explanation", None),
    }
    combined_text = f"{subset_dict['application']} {subset_dict['id']} {subset_dict['logql_query']} {subset_dict['variables']}"
    return {"token_count": count_tokens(combined_text)}


def process_application(dataset, app_name):
    app_dataset = dataset.filter(lambda x: x["application"] == app_name)
    app_dataset_tokens = app_dataset.map(process_token_count)
    total_tokens = sum(app_dataset_tokens["token_count"])
    avg_tokens = (
        total_tokens / len(app_dataset_tokens) if len(app_dataset_tokens) > 0 else 0
    )

    print(f"\nStats for {app_name}:")
    print(f"Total tokens in {app_name} rows: {total_tokens}")
    print(f"Average tokens per {app_name} row: {avg_tokens:.2f}")
    print(f"Number of {app_name} rows: {len(app_dataset_tokens)}")

    return app_dataset_tokens


# Process each application
applications = ["openssh", "openstack", "hdfs"]
app_datasets = {}

for app in applications:
    app_datasets[app] = process_application(dataset, app)

# If you want to calculate overall stats
all_tokens = sum(sum(app_datasets[app]["token_count"]) for app in applications)
all_rows = sum(len(app_datasets[app]) for app in applications)
overall_avg = all_tokens / all_rows if all_rows > 0 else 0

print("\nOverall Stats:")
print(f"Total tokens across all applications: {all_tokens}")
print(f"Average tokens per row across all applications: {overall_avg:.2f}")
print(f"Total number of rows processed: {all_rows}")

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/34 [00:00<?, ? examples/s]


Stats for openssh:
Total tokens in openssh rows: 2607
Average tokens per openssh row: 76.68
Number of openssh rows: 34


Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]


Stats for openstack:
Total tokens in openstack rows: 3753
Average tokens per openstack row: 98.76
Number of openstack rows: 38


Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]


Stats for hdfs:
Total tokens in hdfs rows: 2110
Average tokens per hdfs row: 75.36
Number of hdfs rows: 28

Overall Stats:
Total tokens across all applications: 8470
Average tokens per row across all applications: 84.70
Total number of rows processed: 100


In [125]:
SYSTEM_PROMPT = """
You are a senior data analyst, in-charge of creating datasets for training ML models internally. You approach each step of the data curation flow meticulously.

You are currently dealing with a dataset for natural language questions to logql queries. LogQL is Grafana Loki's Log Query language for searching through observability and log data.

Your current task is to understand existing information about the dataset and different categories in the dataset and add two more columns.

First, here's some information about _all_ the rows.
1. application
   Description: Indicates the software application associated with the log data, in this case "openstack".
2. id
   Description: A unique identifier for each query or question in the dataset.
3. question
   Description: The human-readable question or task that the LogQL query is designed to answer.
4. logql_query
   Description: The actual LogQL (Log Query Language) query used to extract information from the logs.
5. query_explanation
   Description: A detailed explanation of how the LogQL query works and what it does.
6. query_result
   Description: The output or result obtained from executing the LogQL query.
7. category
   Description: The general category or type of operation the query pertains to (e.g., "API Performance and Requests", "Instance Lifecycle Management").
8. log_category_result
   Description: A JSON-like structure containing analysis of the log query, including a 'chain_of_thought' explanation.
9. line_filter
   Description: Indicates whether the query uses single or multiple line filters.
10. label_filter
    Description: Specifies if the query uses single or multiple log stream selectors.
11. metric_category_result
    Description: A JSON-like structure containing analysis of the metric aspects of the query, including categories and a 'chain_of_thought' explanation.
12. metric_category
    Description: Lists the specific metric categories identified in the query (e.g., "unwrapped_range_aggregation", "built_in_range_aggregation").
13. variables
    Description: A list of variables or parameters used in the query, such as time ranges, instance IDs, or specific resources.

You'll be provided with a row of the dataset, templates of the logs the logql query is parsing through and a set of variables for each application that i would want to add.

Your task is to add two new columns to this dataset: row_variables and application_variables.

row_variables are variables specific to a particular row's logql_query. These are elements in a logql_query that can vary, such as time_in_hours or request_method.

application_variables are variables that pertain to an application's logql_query across different rows. These are variables that can vary across all different rows of an application.

You'll receive all the rows of a particular application.

To complete this task, follow these steps:

1. Read each row that will be provided to you. 
   a. Focus on `logql_query` and `query_explanation` fields to understand the log query and what it's fetching.
   b. Focus on `variables` fields to view all the variables the row / application could have. These are human-annotated fields and so are lexically inconsistent but semantically consistent.

2. For each unique application in the dataset, create the application_variables column:
   a. Identify variables that are mentioned in the application_args_vars.
   b. Based on only the information provided, add the variables.
   c. Everything else is a row_variable

2. For each row, create the row_variables column:
   a. Examine the `variables` field to identify variables specific to that query.
   b. Based on `logql_query` and `query_explanation`, if needed, re-write variables.
   c. Don't add unnecessary variables.

4. An application_variable CANNOT be a row_variable. A row_variable CANNOT be an application_variable.

5. If an application_variable isn't in the query, still enforce the request and add it!

6. You will have access to all the variables, make sure you write the `row_variables` in a consistent manner!
"""
# Make sure to include all rows from the original dataset in your output, even if some variables are empty.

In [126]:
USER_PROMPT = """<application>
{APPLICATION}
</application>

<dataset_rows>
{DATASET_ROW}
</dataset_rows>

<log_templates>
{LOG_TEMPLATES}
</log_templates>

<application_args_vars>
{APPLICATION_ARGS}
</application_args_vars>
"""

### LLM Variable Process

In [95]:
from pydantic import BaseModel, Field, field_validator
from typing import List, Dict
import json
from instructor.utils import disable_pydantic_error_url

disable_pydantic_error_url()


class Variables(BaseModel):
    row_id: int
    application: str = Field(
        description="The application whose logs are being processed."
    )
    chain_of_thought: str = Field(
        description="Your step-by-step analysis of the instructions and how you would apply them to find the appropriate variables from the dataset row."
    )
    application_variables: List[str] = Field(
        description="The variables that are pertaining to the entire application."
    )
    row_variables: List[str] = Field(
        description="The variables pertaining to the dataset row. Meaning, the fields that can be changed in the row without fundamentally changing the query itself."
    )

    @field_validator("application")
    @classmethod
    def validate_application(cls, v: str) -> str:
        allowed_applications = {"openssh", "openstack", "hdfs"}
        if v not in allowed_applications:
            raise ValueError(f"application must be one of {allowed_applications}")
        return v

    @field_validator("application_variables")
    @classmethod
    def validate_application_variables(cls, v: List[str], info) -> List[str]:
        application = info.data.get("application")
        if application == "openssh":
            expected = ["hostname", "application"]
        elif application in {"openstack", "hdfs"}:
            expected = ["application"]
        else:
            # This shouldn't happen due to the previous validator, but just in case
            raise ValueError("Invalid application")

        if set(v) != set(expected):
            raise ValueError(
                f"For application '{application}', application_variables must be exactly {expected}"
            )
        return v


def get_var(application: str, row: dict, templates: str, application_vars: List) -> Variables:
    return client.chat.completions.create(
        model="gpt-4o",
        response_model=List[Variables],
        max_retries=2,
        temperature=0.1,
        max_tokens=4096,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {
                "role": "user",
                "content": USER_PROMPT.format(
                    APPLICATION=application,
                    DATASET_ROW=json.dumps(row),
                    LOG_TEMPLATES=templates,
                    APPLICATION_ARGS=application_vars,
                ),
            },
        ],
    )

### Process

In [120]:
columns = ["id", "application", "logql_query", "query_explanation", "variables"]

openstack_dataset = dataset.filter(lambda x: x["application"] == "openstack")
openssh_dataset = dataset.filter(lambda x: x["application"] == "openssh")
hdfs_dataset = dataset.filter(lambda x: x["application"] == "hdfs")

openstack_dict = (
    openstack_dataset.select_columns(columns).to_pandas().to_json(orient="records")
)

openssh_dict = (
    openssh_dataset.select_columns(columns).to_pandas().to_json(orient="records")
)
hdfs_dict = hdfs_dataset.select_columns(columns).to_pandas().to_json(orient="records")

In [78]:
def process_row(generated_vars) -> Dict:
    # Create a subset dictionary with only the required keys
    # subset_dict = {
    #     "id": row.get("id", None),
    #     "application": row.get("application", None),
    #     "logql_query": row.get("logql_query", None),
    #     "query_explanation": row.get("query_explanation", None),
    #     "variables": row.get("variables", None),
    # }

    # Get templates and application_vars from application_dict
    # application_name = subset_dict["application"]
    templates, application_vars = application_dict.get(application, (None, None))

    if templates is not None:
        # Convert templates to CSV string
        templates_csv = templates.to_csv(index=False)

        # Pass the subset_dict to get_var function
        new_variables = get_var(application, dataset_rows, )
        # get_var(application, dataset_rows)
        # print(new_variables)
    else:
        new_variables = None

    # Add new_variables to the original row
    row["row_variables"] = new_variables.row_variables
    row["application_variables"] = new_variables.application_variables

    return row

## Generating Variables

In [138]:
def add_variables(example, vars_dict):
    row_id = example["id"]  # Assuming your dataset has a 'row_id' column
    if row_id in vars_dict:
        v = vars_dict[row_id]
        example["application_variables"] = v.application_variables
        example["row_variables"] = v.row_variables
    else:
        # Handle the case where row_id is not in variables_dict
        example["application_variables"] = []
        example["row_variables"] = []
    return example

**OpenStack**

In [139]:
user_msg = USER_PROMPT.format(
    APPLICATION="openstack",
    DATASET_ROW=openstack_dict,
    LOG_TEMPLATES=openstack_templates_df.to_csv(),
    APPLICATION_ARGS=openstack_application_var,
)
openstack_vars = client.chat.completions.create(
    model="gpt-4o",
    response_model=List[Variables],
    max_retries=2,
    temperature=0.1,
    max_tokens=4096,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_msg},
    ],
)

In [140]:
openstack_variables_dict: Dict[int, Variables] = {v.row_id: v for v in openstack_vars}

In [141]:
openstack_dataset = openstack_dataset.map(
    add_variables, fn_kwargs={"vars_dict": openstack_variables_dict}
)

/home/sidbin/Main-Quests/sauron/dataset-curation/.venv/lib/python3.12/site-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <class '__main__.Variables'>.
  StockPickler.save(self, obj, save_persistent_id)
/home/sidbin/Main-Quests/sauron/dataset-curation/.venv/lib/python3.12/site-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <class '__main__.Variables'>: __main__.Variables has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


Map:   0%|          | 0/38 [00:00<?, ? examples/s]

**OpenSSH**

In [142]:
user_msg = USER_PROMPT.format(
    APPLICATION="openssh",
    DATASET_ROW=openssh_dict,
    LOG_TEMPLATES=openssh_templates_df.to_csv(),
    APPLICATION_ARGS=openssh_application_var,
)
openssh_vars = client.chat.completions.create(
    model="gpt-4o",
    response_model=List[Variables],
    max_retries=2,
    temperature=0.1,
    max_tokens=4096,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_msg},
    ],
)

In [143]:
openssh_variables_dict: Dict[int, Variables] = {v.row_id: v for v in openssh_vars}

In [144]:
openssh_dataset = openssh_dataset.map(
    add_variables, fn_kwargs={"vars_dict": openssh_variables_dict}
)

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

**HDFS**

In [145]:
user_msg = USER_PROMPT.format(
    APPLICATION="hdfs",
    DATASET_ROW=hdfs_dict,
    LOG_TEMPLATES=hdfs_templates_df.to_csv(),
    APPLICATION_ARGS=hdfs_application_var,
)
hdfs_vars = client.chat.completions.create(
    model="gpt-4o",
    response_model=List[Variables],
    max_retries=2,
    temperature=0.1,
    max_tokens=4096,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_msg},
    ],
)

In [146]:
hdfs_variables_dict: Dict[int, Variables] = {v.row_id: v for v in hdfs_vars}

In [148]:
hdfs_dataset = hdfs_dataset.map(
    add_variables, fn_kwargs={"vars_dict": hdfs_variables_dict}
)

/home/sidbin/Main-Quests/sauron/dataset-curation/.venv/lib/python3.12/site-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <class '__main__.Variables'>.
  StockPickler.save(self, obj, save_persistent_id)
/home/sidbin/Main-Quests/sauron/dataset-curation/.venv/lib/python3.12/site-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <class '__main__.Variables'>: __main__.Variables has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


Map:   0%|          | 0/28 [00:00<?, ? examples/s]

In [149]:
from datasets import concatenate_datasets

final_dataset = concatenate_datasets([openstack_dataset, openssh_dataset, hdfs_dataset])
# final_dataset.unique()

In [150]:
final_dataset.save_to_disk("nl-logql-dataset-00")

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

In [151]:
display(final_dataset.to_pandas())

,application,id,question,logql_query,query_explanation,query_result,category,log_category_result,line_filter,label_filter,metric_category_result,metric_category,variables,application_variables,row_variables
0,openstack,2,How long did it take to spawn instance 3edec1e4-9678-4a3a-a21b-a145a4ee5e61 on the hypervisor?,"{application=""openstack"", log_file_type=""nova-compute"", component=""nova.compute.manager""} |= ""3edec1e4-9678-4a3a-a21b-a145a4ee5e61"" |= ""Took"" |= ""seconds to spawn the instance on the hypervisor"" | regexp ""\\[instance: (?P<instance_id>[^\\]]+)\\] Took (?P<spawn_time>\\d+\\.\\d+) seconds to spawn the instance on the hypervisor"" | line_format ""{{.instance_id}} took {{.spawn_time}}""",bla,3edec1e4-9678-4a3a-a21b-a145a4ee5e61 took 20.58,,"{'chain_of_thought': 'Analyzing the query, it contains three label filters: `application=""openstack""`, `log_file_type=""nova-compute""`, `component=""nova.compute.manager""`. Additionally, there are three line filters (`|= ""3edec1e4-9678-4a3a-a21b-a145a4ee5e61""`, `|= ""Took""`, `|= ""seconds to spawn the instance on the hypervisor""`) followed by a regex operation and a line format operation. The presence of three label filters classifies this under multiple log stream selectors, and the presence of more than one line filter classifies it under multiple line filters.', 'label_filter': 'multiple log stream selectors', 'line_filter': 'multiple line filters'}",multiple line filters,multiple log stream selectors,"{'categories': None, 'chain_of_thought': 'The provided query involves log matching and extraction using filters and regexp but does not perform any quantitative aggregations or statistical computations, such as sum, count, or rate on the extracted data. It includes log filters, a regexp pattern for extracting data, and formatting the output using line_format, which suggests manipulation of text rather than numerical computation for metrics. Therefore, there are no metric aggregations present in this query.'}",None,"[instance_id, time_in_sec]",[application],"[instance_id, spawn_time]"
1,openstack,3,What was the total time taken to build instance 3edec1e4-9678-4a3a-a21b-a145a4ee5e61?,"{application=""openstack"", log_file_type=""nova-compute""} |= `3edec1e4-9678-4a3a-a21b-a145a4ee5e61` |= `Took` |= `seconds to build instance` | regexp `\[instance: (?P<instance_id>[^\]]+)\] Took (?P<build_time>\d+\.\d+) seconds to build instance` | line_format `{{.build_time}}`","1. {application=""openstack"", log_file_type=""nova-compute""}\nFetch all log lines matching label filters.\n2. <expr> |= `3edec1e4-9678-4a3a-a21b-a145a4ee5e61`\nReturn log lines that contain string 3edec1e4-9678-4a3a-a21b-a145a4ee5e61.\n\n3. <expr> |= `Took`\nReturn log lines that contain string Took.\n\n4. <expr> |= `seconds to build instance`\nReturn log lines that contain string seconds to build instance.\n\n5. <expr> | regexp `\[instance: (?P<instance_id>[^\]]+)\] Took (?P<build_time>\d+\.\d+) seconds to build instance`\nThe regexp parser takes a single parameter | regexp ""<re>"" which is the regular expression using the Golang RE2 syntax. The regular expression must contain a least one named sub-match (e.g (?P<name>re)), each sub-match will extract a different label. The expression matches the structure of a log line. The extracted labels can be used in label filter expressions and used as values for a range aggregation via the unwrap operation.\n\n6. <expr> | line_format `{{.buil...",21.38,,"{'chain_of_thought': 'The user query submits using multiple label filters: `application='openstack'`, `log_file_type='nova-compute'`. There are multiple line filters used sequentially: `|= '3edec1e4-9678-4a3a-a21b-a145a4ee5e61'`, `|= 'Took'`, `|= 'seconds to build instance'`, `| regexp '\[instance: (?P<instance_id>[^\]]+)\d+] Took (?P<build_time>\d+.\d+) seconds to build instance'`. By definition, using several different types of line filters suggests it falls under 'multiple line filters'. For labels, using mult